## Scraping Finans Danmark boligstatistik


In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import time
import re
import requests
import pandas as pd

In [5]:
# Set up the driver and navigate to the URL
# Set up the driver and navigate to the URL
url = 'https://rkr.statistikbank.dk/statbank5a/SelectVarVal/Define.asp?MainTable=BM010&PLanguage=0&PXSId=0&wsid=cflist'
driver = webdriver.Chrome()  # Ensure you have the ChromeDriver set up correctly
driver.get(url, headers={'name': 'Lea Lund Jeppesen', 'email':'snk956@alumni.ku.dk'})
time.sleep(5)


#### DROPDOWN 1: Choosing only "Kommuner" ###
# Locate the dropdown element using its name attribute
dropdown1 = driver.find_element(By.NAME, 'grouping1')

# Use the Select class to interact with the dropdown
select = Select(dropdown1)

# Select the "Kommuner" option by its visible text
select.select_by_visible_text('Kommuner')


#### DROPDOWN 2: Choosing only "Parcel-/rækkehus" ###
# Locate the dropdown element using its name attribute
dropdown2 = driver.find_element(By.NAME, 'var2')

# Use the Select class to interact with the dropdown
select = Select(dropdown2)

# Select the "Parcel-/rækkehus" option by its value
select.select_by_value('1')


#### DROPDOWN 3: Choosing only "Realiseringspris" ###
dropdown3 = driver.find_element(By.NAME, 'var3')

# Use the Select class to interact with the dropdown
select = Select(dropdown3)

# Select the "Realiserings" option by its value
select.select_by_value('REAL')


#### DROPDOWN 4: Choosing all years and quarters ###
# Locate the dropdown element using its name attribute
dropdown4 = driver.find_element(By.NAME, 'grouping4')

# Use the Select class to interact with the dropdown
select = Select(dropdown4)

# Select the "Markér alle" option by its visible text
select.select_by_visible_text('Markér alle')


### SELECT "Vis tabel" ###
show_table = driver.find_element(By.NAME, 'Forward')

# Click on the button to show the table
show_table.click()

# driver.close()

In [28]:
data = []

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table', id='pxtable')

rows = table.find_all('tr')

for row in rows:
    columns = row.find_all(['td', 'th'])
    column_data = [col.text for col in columns]
    data.append(column_data)

# Converting our data into a DataFrame
df = pd.DataFrame(data[2:], columns=data[1])  # Assuming the first row contains the column headers

#Drop the two forst rows as they are irrelevant
df = df.drop(df.index[:2])

#Reset index
df=df.reset_index(drop=True)

#Print table
df

#Clean up the table. First, we remove the quarters in the column title 
df.columns = [column[:-2] for column in df.columns]
df

,,1992,1992,1992,1992,1993,1993,1993,1993,1994,...,2020,2021,2021,2021,2021,2022,2022,2022,2022,2023
0,København,5 355,5 451,5 374,5 171,5 068,5 186,5 609,5 519,5 503,...,42 173,45 636,46 966,49 455,47 726,48 374,50 514,48 847,45 816,45 614
1,Frederiksberg,5 183,5 626,6 980,6 686,5 537,7 047,6 651,6 174,6 558,...,64 237,63 713,74 204,73 317,77 872,80 966,90 010,76 058,59 137,72 368
2,Dragør,6 251,6 472,6 364,6 246,6 158,6 438,6 573,6 261,5 969,...,31 565,35 614,39 863,37 155,38 977,39 221,39 422,40 250,32 917,32 168
3,Tårnby,5 747,6 003,6 075,5 856,5 275,5 860,5 976,5 916,6 196,...,29 819,32 239,36 583,36 839,34 563,35 810,35 743,33 920,29 953,31 204
4,Albertslund,5 240,5 408,5 417,5 341,5 130,4 773,5 746,5 492,5 616,...,22 304,24 771,23 988,23 571,24 818,24 458,23 885,24 745,23 574,23 312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Morsø,2 706,2 754,2 702,2 571,2 607,2 794,2 791,2 571,2 694,...,5 758,5 106,5 134,6 400,4 937,6 038,4 910,4 414,5 602,4 900
94,Rebild,3 040,2 901,2 845,2 772,2 710,2 881,3 066,3 208,3 093,...,10 406,11 561,11 336,9 999,10 409,11 256,12 467,10 597,11 993,10 684
95,Thisted,2 750,2 646,2 597,2 485,2 874,2 956,3 038,3 252,3 074,...,7 353,7 187,7 675,7 327,7 279,7 021,7 834,7 575,6 949,7 607
96,Vesthimmerlands,2 802,2 587,2 529,2 730,2 766,2 764,2 924,2 964,3 262,...,6 749,7 554,6 744,7 454,7 255,6 605,6 850,7 624,6 638,7 343


In [29]:


def same_merge(x): return ','.join(x[x.notnull()].astype(str))

df_new=df.groupby(level=0, axis=1).apply(lambda x: x.apply(same_merge, axis=1))

df_new



,,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,København,"5 355,5 451,5 374,5 171","5 068,5 186,5 609,5 519","5 503,5 885,5 621,5 328","5 675,5 924,6 014,6 299","6 764,6 991,7 210,7 227","7 670,8 263,8 234,8 520","9 201,9 275,9 606,9 762","9 925,10 590,10 479,11 078","10 637,11 422,11 737,12 562",...,"25 429,26 568,27 162,27 951","28 418,29 930,30 276,29 875","31 561,31 159,32 114,30 752","32 044,33 937,34 306,34 013","36 761,35 334,35 701,37 494","36 347,36 170,37 003,38 894","37 680,38 803,40 458,42 173","45 636,46 966,49 455,47 726","48 374,50 514,48 847,45 816",45 614
1,Frederiksberg,"5 183,5 626,6 980,6 686","5 537,7 047,6 651,6 174","6 558,6 496,7 283,6 955","7 607,7 165,8 643,8 477","8 174,8 311,8 964,8 772","9 346,9 861,9 806,11 098","10 158,11 822,12 685,13 918","13 288,12 046,10 594,13 914","14 408,14 873,14 358,14 398",...,"38 057,35 275,36 666,38 240","39 828,43 955,40 725,43 772","35 664,44 624,42 008,42 545","46 969,43 342,52 597,53 700","49 203,50 998,49 540,49 267","48 487,50 732,48 625,53 979","47 843,52 770,54 290,64 237","63 713,74 204,73 317,77 872","80 966,90 010,76 058,59 137",72 368
2,Dragør,"6 251,6 472,6 364,6 246","6 158,6 438,6 573,6 261","5 969,6 398,6 272,6 229","6 983,6 703,6 727,7 197","6 739,8 059,8 108,8 375","8 830,9 208,9 129,10 035","9 603,10 515,12 019,11 596","12 021,11 485,12 332,11 973","11 910,12 342,12 966,13 220",...,"24 022,22 705,23 788,24 293","23 645,26 134,26 210,26 137","26 962,27 565,25 777,27 262","25 850,27 822,27 944,28 602","30 477,28 407,28 044,29 931","28 374,29 640,29 270,31 196","29 128,32 128,33 646,31 565","35 614,39 863,37 155,38 977","39 221,39 422,40 250,32 917",32 168
3,Tårnby,"5 747,6 003,6 075,5 856","5 275,5 860,5 976,5 916","6 196,6 167,6 113,5 968","6 172,6 299,6 316,6 911","6 727,6 933,7 277,7 523","7 458,8 474,8 891,9 189","9 153,9 758,9 679,9 878","10 095,10 506,10 608,10 738","9 645,11 759,11 454,12 004",...,"20 155,21 159,22 462,21 727","22 256,23 639,23 595,23 417","25 013,24 704,24 287,25 927","25 551,27 989,27 506,27 322","27 121,27 916,27 943,27 824","28 288,28 657,28 114,29 438","28 553,28 693,29 363,29 819","32 239,36 583,36 839,34 563","35 810,35 743,33 920,29 953",31 204
4,Albertslund,"5 240,5 408,5 417,5 341","5 130,4 773,5 746,5 492","5 616,5 235,5 475,5 455","6 318,6 160,6 197,6 238","6 571,6 568,7 248,7 574","7 718,8 260,7 797,8 729","7 602,9 026,8 272,8 743","9 179,9 246,9 845,9 993","9 927,10 457,10 544,9 839",...,"15 412,16 657,16 212,16 512","16 617,17 284,17 749,17 595","17 802,18 469,19 612,18 988","19 830,20 553,19 403,19 643","20 223,20 710,20 666,21 841","20 454,21 093,20 043,21 112","21 970,21 426,22 015,22 304","24 771,23 988,23 571,24 818","24 458,23 885,24 745,23 574",23 312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Morsø,"2 706,2 754,2 702,2 571","2 607,2 794,2 791,2 571","2 694,2 898,2 666,2 578","2 621,2 696,2 877,3 030","2 986,2 943,2 750,3 046","2 969,3 215,2 938,3 249","3 031,3 216,3 125,3 100","3 261,2 902,3 344,4 099","3 092,4 120,3 847,4 060",...,"5 547,4 623,4 706,5 703","6 123,5 275,5 784,4 736","4 505,5 859,5 523,5 041","6 157,5 459,4 898,5 778","5 018,4 656,5 431,4 950","5 511,4 893,5 501,4 972","4 911,5 584,4 757,5 758","5 106,5 134,6 400,4 937","6 038,4 910,4 414,5 602",4 900
94,Rebild,"3 040,2 901,2 845,2 772","2 710,2 881,3 066,3 208","3 093,3 302,3 558,3 404","3 471,3 538,3 378,3 550","3 389,3 687,3 571,3 784","3 658,3 888,3 713,3 832","4 244,4 173,4 284,4 169","4 619,4 794,5 241,4 711","4 436,4 807,5 305,5 051",...,"9 093,8 744,8 038,9 353","8 467,8 941,8 272,8 333","8 762,8 843,9 783,8 584","9 358,9 476,8 160,9 333","8 931,9 795,9 192,9 416","10 170,9 951,9 669,10 810","10 539,9 784,9 887,10 406","11 561,11 336,9 999,10 409","11 256,12 467,10 597,11 993",10 684
95,Thisted,"2 750,2 646,2 597,2 485","2 874,2 956,3 038,3 252","3 074,3 171,3 094,3 139","3 092,3 105,3 190,3 277","3 350,3 437,3 560,3 566","3 612,